In [ ]:
%pylab inline

In [ ]:
import yaml
import h5py
import numpy as np

import pica

In [ ]:
print (pica.__version__)

In [ ]:
%pwd

In [ ]:
input_filename = f'testbeam1'
SIM = pica.ICSSimulation( input_filename )
SIM.run()
#SIM.read()


In [ ]:
SIM.K_photon.shape

K0,K1,K2,K3       = SIM.K_photon   * (SIM.momentum_scale/1e9) # always in GeV for analysis
X0,X1,X2,X3       = SIM.X_photon
W                 = SIM.W_photon
_,S1,S2,S3        = SIM.S_photon # in x-z plane

P0,P1,P2,P3       = SIM.P_electron * (SIM.momentum_scale/1e9) # always in GeV for analysis


PolDegree         = SIM.LinearPolarizationDegree  # photon linear polarization degree
TiltAngle         = 0.5*np.arctan2(S2,S1)         # polarization tilt angle

In [ ]:
with open( input_filename + '.yml', 'r' ) as stream:
    input_dict = yaml.load(stream, Loader=yaml.SafeLoader)
        
    beam_charge      = float( input_dict['beam']['beam_charge'])
    number_electrons = int( beam_charge / 1.60217653e-19)

#     sampling         = input_dict['control']['sampling']
    
    w0               = float(input_dict['laser']['w0'])
    sigmaX           = float(input_dict['beam']['sigmaX'])
    sigmaY           = float(input_dict['beam']['sigmaY'])
    
    baseline         = float(input_dict['beam']['baseline'])
    
omegamin,omegamax    = np.asarray(SIM.omega_detector)/1e9  # in GeV
thetamin,thetamax    = np.asarray(SIM.theta_detector)*1e6  # in µrad

In [ ]:
print (f'beam charge                              : {SIM.beam_charge*1e12:.24} pC')
print (f'number of beam electrons                 : {SIM.number_electrons:.4e}')
print ('-'*42)
print (f'max  single photon weight                : {amax(W):.4g}')
print (f'mean single photon weight                : {mean(W):.4g}')
print (f'total photon weight                      : {sum(W):.4g}')
print (f'total photon weight per incident electron: {sum(W)/number_electrons:.4g}')
print (f'total macro photon number                : {len(W)}')
print ('-'*42)

# Photons at Source
## Photon Energy-Angle

In [ ]:
figure(figsize=(10,6), constrained_layout=True)

bbins=(linspace(omegamin,omegamax,100),linspace(thetamin,thetamax,102))

subplot(111)
hist2d(K0,sqrt(K1**2+K2**2)/K3*1e6,bins=bbins,weights=W,rasterized=True,cmap=cm.magma);
colorbar(label=r'$\frac{d^2N}{d\omega \, d\cos\theta}$',shrink=0.7)
xlabel(r'$\omega$ (GeV)')
ylabel(r'$\theta$ ($\mu$rad)');


## Photon Transverse Momentum

In [ ]:
figure(figsize=(10,6), constrained_layout=True)

subplot(111,aspect=1)
hist2d(K1*1e6,K2*1e6,bins=(50,50),weights=W,rasterized=True,cmap=cm.magma);
colorbar(label=r'$\frac{d^2N}{d^2\vec k_\perp}$', shrink=0.7)

xlabel(r'$k_x$ (keV)')
ylabel(r'$k_y$ (keV)');


## Photon Angular Distribution

In [ ]:
figure(figsize=(10,6), constrained_layout=True)

subplot(111,aspect=1)
hist2d(K1/K3*1e6,K2/K3*1e6,bins=(70,72),weights=W,rasterized=True,cmap=cm.magma);
colorbar(label=r'$\frac{d^2N}{d\theta_x \, d\theta_y}$',shrink=0.7)

xlabel(r'$\theta_x$ (µrad)')
ylabel(r'$\theta_y$ (µrad)');


## Photon Radial Source Spectrum

In [ ]:
figure(figsize=(10,6), constrained_layout=True)

bbins=(linspace(omegamin,omegamax,100),linspace(0,2*sigmaX,51))

subplot(111)
hist2d(K0,sqrt(X1**2+X2**2),bins=bbins,weights=W/sqrt(X1**2+X2**2),rasterized=True,cmap=cm.magma);
colorbar(shrink=0.7)
xlabel(r'$\omega$ (GeV)')
ylabel(r'$r$ ($\mu$m)');
ylim(0,2*sigmaX)


## Photon Source Size

In [ ]:
figure(figsize=(10,6), constrained_layout=True)

bbins=(linspace(-2*sigmaX,2*sigmaX,99),linspace(-2*sigmaY,2*sigmaY,100))

subplot(111,aspect=1)
hist2d(X1,X2,bins=bbins,weights=W,rasterized=True,cmap=cm.magma);
colorbar(shrink=0.7)

xlabel(r'$x@$source (micron)')
ylabel(r'$y@$source (micron)');


# Scattered Electron Momenta

In [ ]:
fig,ax=subplots(1,3,figsize=(20,6), constrained_layout=True)
bbins=(50,50)

ax[0].hist2d(P3,P1*1e6,bins=bbins,weights=W,rasterized=True,cmap=cm.magma);
ax[1].hist2d(P3,P2*1e6,bins=bbins,weights=W,rasterized=True,cmap=cm.magma);
ax[2].hist2d(P1*1e6,P2*1e6,bins=bbins,weights=W,rasterized=True,cmap=cm.magma);


ax[0].set_xlabel(r'electron $p_z$ (GeV)')
ax[1].set_xlabel(r'electron $p_z$ (GeV)')
ax[2].set_xlabel(r'electron $p_x$ (keV)')

ax[0].set_ylabel(r'electron $p_x$ (keV)')
ax[1].set_ylabel(r'electron $p_y$ (keV)')
ax[2].set_ylabel(r'electron $p_y$ (keV)');


# Photons at IP

In [ ]:
xoffset = SIM.xoffset
yoffset = SIM.yoffset

X1_IP = X1+xoffset
X2_IP = X2+yoffset

R_IP  = sqrt(X1_IP**2 + X2_IP**2)

In [ ]:
spotsize  = 15  # micron
select    = R_IP < spotsize


print ('-'*82)
print ( input_filename)
print ('-'*82)
print (f'IP-laser spot size                  : {spotsize:.2f} micron')
print (f'baseline distance                   : {baseline/1e6:.2f} metres')
print ('-'*82)


print (f'min macrophoton weight              : {amin(W):.6g}')
print (f'max macrophoton weight              : {amax(W):.6g}')
print (f'macrophotons in spot                : {sum( select ):.0f}' )
print (f'total photon weight in spot         : {sum(W[select ]):.1f}' )
print (f'photon density                      : {sum(W[select ] ) /(np.pi*spotsize**2):.4f} photons/micron^2')

print ('='*82)



## Photon Energy

In [ ]:
figure(figsize=(10,6), constrained_layout=True)
ax=subplot(111,aspect=1)

scatter(X1_IP, X2_IP, c=K0, marker='.', lw=0, s=5*W, cmap=cm.Spectral_r);
colorbar(label='$\omega$ (GeV)', shrink=0.7)

xlabel(r'$x$ (micron)')
ylabel(r'$y$ (micron)')
ax.add_patch(Circle((0,0),spotsize,color='0.5',fill=False));


## Photon Stokes Parameter S1

In [ ]:
figure(figsize=(10,6), constrained_layout=True)
ax=subplot(111,aspect=1)
scatter(X1_IP, X2_IP, c=S1, marker='.',lw=0,s=5*W,vmin=-1,vmax=1,cmap=cm.PuOr);

colorbar(label='Stokes parameter $S_1$', shrink=0.7)
xlabel(r'$x$ (micron)')
ylabel(r'$y$ (micron)');

ax.add_patch(Circle((0,0),spotsize,color='0.5',fill=False));


## Photon Linear Polarization Degree

In [ ]:
figure(figsize=(10,6), constrained_layout=True)
ax=subplot(111,aspect=1)
scatter(X1_IP, X2_IP, c=PolDegree, marker='.',lw=0,s=5*W,vmin=-1,vmax=1,cmap=cm.Reds);

colorbar(label='Stokes parameter $S_1$', shrink=0.7)
xlabel(r'$x$ (micron)')
ylabel(r'$y$ (micron)');

ax.add_patch(Circle((0,0),spotsize,color='0.5',fill=False));


## Photon Polarization Tilt Angle

In [ ]:
figure(figsize=(10,6), constrained_layout=True)
ax=subplot(111,aspect=1)
scatter(X1_IP, X2_IP, c=TiltAngle*180/pi, marker='.',vmin=-90, vmax=90, lw=0,s=5*W,cmap=cm.twilight_shifted);

colorbar(label='Photon polarization tilt angle (deg)', shrink=0.7)
xlabel(r'$x$ (micron)')
ylabel(r'$y$ (micron)');

ax.add_patch(Circle((0,0),spotsize,color='0.5',fill=False));
